#### This page shows features that may provide poor volume rendering performance on slow devices. 
#### Using anti-aliasing, high-DPI, and reflective volume rendering modes all impact performance.

This example is based off of [https://niivue.com/demos/features/vox.tradeoffs.html](https://niivue.com/demos/features/vox.tradeoffs.html).

# Import modules

In [ ]:
import json
import pathlib

import ipywidgets as widgets

import ipyniivue
from ipyniivue import NiiVue, download_dataset
from ipyniivue.constants import ShowRender, SliceType

# Download required data

In [ ]:
DATA_FOLDER = pathlib.Path(ipyniivue.__file__).parent / "images"

download_dataset(
    files=["mni152.nii.gz", "aal.nii.gz", "stats.nv_demo_mskd.nii.gz", "aal.json"],
)

# Create an instance of NiiVue and load the volumes

In [ ]:
nv = NiiVue(
    back_color=(1, 1, 1, 1),
    show_3d_crosshair=True,
    multiplanar_show_render=ShowRender.ALWAYS,
    is_colorbar=True,
    slice_type=SliceType.MULTIPLANAR,
)

volume_list = [
    {"path": DATA_FOLDER / "mni152.nii.gz"},
    {"path": DATA_FOLDER / "aal.nii.gz"},
    {
        "path": DATA_FOLDER / "stats.nv_demo_mskd.nii.gz",
        "colormap": "warm",
        "colormap_negative": "winter",
        "frame4D": 1,
        "cal_min": 3.3641,
        "cal_max": 6,
    },
]

nv.load_volumes(volume_list)

# Hide colorbars for volumes 0 and 1
nv.volumes[0].colorbar_visible = False
nv.volumes[1].colorbar_visible = False

# Set colormap on volume load

In [ ]:
@nv.on_image_loaded
def handle_image_loaded(volume):
    """Update colormap label on image loaded."""
    if volume.name == "aal.nii.gz":
        with open(DATA_FOLDER / "aal.json") as f:
            cmap = json.load(f)
        volume.set_colormap_label(cmap)

# Create location status bar

In [ ]:
status_bar = widgets.Label(value="")


@nv.on_location_change
def handle_location_change(location):
    """Display location."""
    status_bar.value = location["string"]

# Create extra widgets

In [ ]:
aa_check = widgets.Checkbox(value=True, description="AntiAlias")
dpi_check = widgets.Checkbox(value=True, description="HighDPI")
render_mode = widgets.Dropdown(
    options=[
        ("slices", "-1"),
        ("matte", "0"),
        ("low", "0.3"),
        ("medium", "0.6"),
        ("high", "1.0"),
    ],
    value="0.6",
    description="Render Mode",
)
atlas_slider = widgets.IntSlider(
    value=64, min=0, max=255, description="Atlas", orientation="horizontal"
)
stat_slider = widgets.IntSlider(
    value=255, min=0, max=255, description="Statistics", orientation="horizontal"
)


def on_dpi_check_change(change):
    """Set high resolution capable."""
    nv.set_high_resolution_capable(change["new"])


def on_render_mode_change(change):
    """Set volume render illumination."""
    nv.set_volume_render_illumination(float(change["new"]))


def on_atlas_slider_change(change):
    """Set 2nd volume opacity."""
    nv.volumes[1].opacity = change["new"] / 255.0


def on_stat_slider_change(change):
    """Set 3rd volume opacity."""
    nv.volumes[2].opacity = change["new"] / 255.0


def on_aa_check_change(change):
    """Update is_anti_alias option."""
    nv.is_anti_alias = change["new"]


dpi_check.observe(on_dpi_check_change, names="value")
render_mode.observe(on_render_mode_change, names="value")
atlas_slider.observe(on_atlas_slider_change, names="value")
stat_slider.observe(on_stat_slider_change, names="value")
aa_check.observe(on_aa_check_change, names="value")

# Display all

In [ ]:
on_aa_check_change({"new": aa_check.value})
on_dpi_check_change({"new": dpi_check.value})
on_render_mode_change({"new": render_mode.value})
on_atlas_slider_change({"new": atlas_slider.value})
on_stat_slider_change({"new": stat_slider.value})

widgets.VBox(
    [
        widgets.HBox([aa_check, dpi_check, render_mode]),
        widgets.HBox([atlas_slider, stat_slider]),
        nv,
        status_bar,
    ]
)